In [2]:
import os
import sys
from glob import glob

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import desispec
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

from desiutil.log import get_logger, DEBUG

from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *

from timedomain.sp_utils import SkyPortal as sp
import requests
import datetime
import heapq
import time
import copy
import numpy
from astropy.time import Time
from astropy.table import Table, vstack, unique, SortedArray

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython import display

In [3]:
#SkyPortal token:
secret_file = "/global/cfs/cdirs/desi/science/td/secrets/desidiff_sp.txt"
with open(secret_file, 'r') as file:
    token = file.read().replace('\n', '')
headers = {'Authorization': f'token {token}'}

filter_name = 'DESIDIFF'

pdf1 = PdfPages("All_plots.pdf")

In [4]:
# read in and store in one place all the fibermap information in the spectra files
night = 20210608
current_filename = '/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/spectra-6-1097-thru20210608.fits'

tile = current_filename.split('/')[-3]
petal = current_filename.split('-')[1]
### daily_spectra, the precursor to current spectra, before coadding, to select night, unique target ids, and individual target ids
### spectra.select functionality will not work once coadded with coaddition.coadd_cameras
daily_spectra = ((read_spectra(current_filename)).select(nights = night))

#unique_targetids = numpy.unique(daily_spectra.fibermap['TARGETID'][numpy.where(daily_spectra.fibermap['OBJTYPE']=='TGT')])
#print(len(unique_targetids))

table = Table.read(current_filename, format='fits',hdu=1, memmap=True) 
##### DAVE'S ADDITION ##############
targetcols = [i for i in table.colnames if i[-7:] =='_TARGET']
nonzerocheck = [True in k for k in [[j != 0 for j in table[targetcols][i]] for i in range(len(table))]]
#a really ugly line, basically generates a list of bools, 
#True if there is at least one nonzero element in all columns ending in _TARGET
table.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
#This gets the index of all False values from the previous list and removes those rows
table = table['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
######## END DAVE'S ADDITION ############
table = table[numpy.logical_and(table['OBJTYPE']=='TGT', table['FIBERSTATUS']==0)]



INFO:spectra.py:291:read_spectra: iotime 0.523 sec to read spectra-6-1097-thru20210608.fits at 2022-07-05T14:47:19.898328


In [35]:
ra_dec = (table[table['TARGETID']==t]['TARGET_RA'][0], table[table['TARGETID']==t]['TARGET_DEC'][0])
print(ra_dec)

229.14264392094805


In [5]:
ref_filename = '/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits'

if ref_filename.split('/')[-6] == 'fuji' or ref_filename.split('/')[-6] == 'guadalupe':
    prev_spectra = read_spectra(ref_filename)
    


INFO:spectra.py:291:read_spectra: iotime 0.497 sec to read coadd-6-1097-thru20210608.fits at 2022-07-05T14:47:30.107542


In [6]:
gen = (t for t in daily_spectra.target_ids()[numpy.where(daily_spectra.fibermap['OBJTYPE']=='TGT')] if t in prev_spectra.target_ids()[numpy.where(prev_spectra.fibermap['OBJTYPE']=='TGT')])
for t in gen:
    current_spectra = desispec.coaddition.coadd_cameras(daily_spectra.select(targets = t))
    ref_spectra = desispec.coaddition.coadd_cameras(prev_spectra.select(targets = t))
    norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)

In [7]:
for tid in daily_spectra.target_ids()[numpy.where(daily_spectra.fibermap['OBJTYPE']=='TGT')]:
    if tid in prev_spectra.target_ids()[numpy.where(prev_spectra.fibermap['OBJTYPE']=='TGT')]:
        ### copy.deepcopy() is deprecated as memory expensive
        current_spectra = desispec.coaddition.coadd_cameras(daily_spectra.select(targets = tid))
        
        ref_spectra = desispec.coaddition.coadd_cameras(prev_spectra.select(targets = tid))
        
        norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)
        diff_spectra = copy.deepcopy(current_spectra)
        print(diff_spectra.num_targets())
        #current_spectra.flux['brz'] = current_spectra.flux['brz']/norm
        
        #current_spectra.ivar['brz'] = current_spectra.ivar['brz']*norm**2  
        
        # subtraction of current and reference fluxes
        #dif_spectra.flux = current_spectra.flux['brz'] - ref_current_spectra.flux['brz']
        
        # summation of current and reference masks        
        #dif_spectra.mask['brz'] = current_spectra.mask['brz'] + ref_current_spectra.mask['brz']

        # inverted summation of current and spectra inverse variance
        ### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
        #dif_spectra.ivar['brz'] = 1./(1./current_spectra.ivar['brz'] + 1./ref_current_spectra.ivar['brz'])
        
        # mean-subtracted difference
        #clippy = clipmean(dif_spectra.flux['brz'],diff_ivar,diff_mask)
 
  

KeyboardInterrupt: 

In [7]:
#print(tid)
print(t)

39627891209668033


In [6]:

current_spectra = desispec.coaddition.coadd_cameras(daily_spectra.select(targets = t))

ref_spectra = desispec.coaddition.coadd_cameras(prev_spectra.select(targets = t))

norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)

current_spectra.flux['brz'] = current_spectra.flux['brz']/norm

current_spectra.ivar['brz'] = current_spectra.ivar['brz']*norm**2  
### copy.deepcopy() is deprecated as memory expensive
dif_spectra = copy.deepcopy(current_spectra)
print(dif_spectra.num_targets())

# subtraction of current and reference fluxes
dif_spectra.flux['brz'] = current_spectra.flux['brz'] - ref_spectra.flux['brz']

# summation of current and reference masks        
dif_spectra.mask['brz'] = current_spectra.mask['brz'] + ref_spectra.mask['brz']

# inverted summation of current and spectra inverse variance
### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
dif_spectra.ivar['brz'] = 1./(1./current_spectra.ivar['brz'] + 1./ref_spectra.ivar['brz'])



1


/tmp/ipykernel_63434/262105920.py:22: RuntimeWarning: divide by zero encountered in true_divide
  dif_spectra.ivar['brz'] = 1./(1./current_spectra.ivar['brz'] + 1./ref_spectra.ivar['brz'])


In [7]:
# mean-subtracted difference
dif_flux_clipped = clipmean(dif_spectra.flux, dif_spectra.ivar, dif_spectra.mask)

In [47]:
print(dif_flux_clipped)
print(dif_flux_clipped['brz'])
print(dif_flux_clipped['brz'][0])

{'brz': array([[-1.43442289, -1.371912  , -0.13772431, ...,  0.14851908,
         0.11883302,  0.04833728]])}
[[-1.43442289 -1.371912   -0.13772431 ...  0.14851908  0.11883302
   0.04833728]]
[-1.43442289 -1.371912   -0.13772431 ...  0.14851908  0.11883302
  0.04833728]


In [8]:
# filters 

# Difference spectrum may have broadband signal
perband_filter = perband_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask)


In [140]:
def perconv_SN(wave, dif_flux_clipped, ivar, mask, ncon=5, nsig=10):
    ncon = 5
    nsig = 10
    newmask = dict(dif_spectra.mask)
    print(newmask)

    newston = dict(dif_flux_clipped)
    print(newston)

    newy = dict(dif_flux_clipped)
    print(newy)

    conk = numpy.zeros(ncon) + 1.

    nTrue = 0
    for b in newston.keys():
        # mask lines that are smaller than the ncov
        f_logic = numpy.abs(dif_flux_clipped[b]*numpy.sqrt(dif_spectra.ivar[b])) > 8
        index=0
        for _, g in itertools.groupby(f_logic):
            ng  = sum(1 for _ in g)
            print(_, ng)
            for x in _:
                if x and ng<=3:
                    newy[b][index:index+ng] = 0
            index=index+ng


        newston[b] = numpy.array(numpy.convolve(newy[b][0], conk, mode='valid') / numpy.sqrt(numpy.convolve(1/dif_spectra.ivar[b][0], conk, mode='valid')))
        newmask[b] = numpy.array(numpy.convolve(dif_spectra.mask[b][0], conk, mode='valid'))

        w = numpy.where(newmask[b] == 0)[0]
        print(newston[b].size)
        print(newston[b][w])
        for gb0,_ in itertools.groupby(newston[b][w] > nsig):
            if gb0:
                nTrue=nTrue+1
    print(nTrue)


{'brz': array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint32)}
{'brz': array([[-1.43442289, -1.371912  , -0.13772431, ...,  0.14851908,
         0.11883302,  0.04833728]])}
{'brz': array([[-1.43442289, -1.371912  , -0.13772431, ...,  0.14851908,
         0.11883302,  0.04833728]])}
[False False False ... False False False] 1
7777
[-0.3341374  -0.33886909 -0.30655877 ...  0.57731738  0.63376811
  0.7049891 ]
0


/tmp/ipykernel_46155/2070089250.py:29: RuntimeWarning: divide by zero encountered in true_divide
  newston[b] = numpy.array(numpy.convolve(newy[b][0], conk, mode='valid') / numpy.sqrt(numpy.convolve(1/dif_spectra.ivar[b][0], conk, mode='valid')))


In [9]:
def test_perconv_SN(y, ivar, mask, ncon=5, nsig=10):
    newmask = dict(mask)
    newston = dict(y)
    newy = dict(y)
    conk = numpy.zeros(ncon) + 1.

    nTrue = 0
    for b in newston.keys():
        # mask lines that are smaller than the ncov

        f_logic = numpy.abs(y[b]*numpy.sqrt(ivar[b])) > 8
        index=0
        for _, g in itertools.groupby(f_logic):
            ng  = sum(1 for _ in g)
            for x in _:
                if x and ng<=3:
                    newy[b][index:index+ng] = 0
                index=index+ng

        newston[b]=numpy.array(numpy.convolve(newy[b][0], conk, mode='valid') / numpy.sqrt(numpy.convolve(1/ivar[b][0], conk, mode='valid')))
        newmask[b] = numpy.array(numpy.convolve(mask[b][0], conk, mode='valid'))

        w = numpy.where(newmask[b] == 0)[0]
        for gb0,_ in itertools.groupby(newston[b][w] > nsig):
            if gb0:
                nTrue=nTrue+1
    return nTrue

In [10]:

# fractional increase
perband_inc = perband_increase(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ref_spectra.flux, ref_spectra.ivar, ref_spectra.mask)

# Difference spectrum may have high-frequency signal
perres_filter_broad = test_perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 100, nsig = 17)

perres_filter_narrow = test_perconv_SN(dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, ncon = 5, nsig = 10)



/tmp/ipykernel_63434/3042030317.py:20: RuntimeWarning: divide by zero encountered in true_divide
  newston[b]=numpy.array(numpy.convolve(newy[b][0], conk, mode='valid') / numpy.sqrt(numpy.convolve(1/ivar[b][0], conk, mode='valid')))


In [11]:
import h5py

#redrock = Table.read(current_filename.replace('spectra','redrock'), format='fits',hdu=1, memmap=True)['TARGETID','Z','ZERR','ZWARN','SPECTYPE']


In [12]:
print(current_filename)
current_filename = current_filename.replace('spectra','redrock')
print(current_filename)
current_filename = current_filename.replace('fits','h5')
print(current_filename)

/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/spectra-6-1097-thru20210608.fits
/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/redrock-6-1097-thru20210608.fits
/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/redrock-6-1097-thru20210608.h5


In [13]:
with h5py.File(current_filename, "r") as f:
    zinfo = f['zfit']['39627867075642519']['zfit'][0]['z']

In [108]:
print(zinfo)

0.5941702367437508


In [15]:
def test_line_finder(wave, flux,ivar,mask,z):
    c = 2.99e5 #km/s

    lines = ['Halpha','Hbeta', 'Hgamma','HeII4686','OIII5007','NIII','SII','OIII4959']
    restwavelengths = [6562,4861,4340,4686,5007,4100, 6732,4959]
    
    key = list(wave.keys())[0] #if data is coadded, should be the first key
        
    
    wave, flux, mask, ivar = wave[key], flux[key][0], mask[key][0], ivar[key][0]
    
    
    HB_center = list(abs(wave-4861.4)).index(min(abs(wave - 4861.4)))
    HBmask = mask[HB_center - 400:HB_center + 400]
    HBroi = ma.masked_array(wave[HB_center - 400:HB_center + 400], HBmask)
    HBflux = ma.masked_array(flux[HB_center - 400:HB_center + 400], HBmask)
    HBivar = ivar[HB_center - 400:HB_center + 400]
    HBsigma = 1/np.sqrt(HBivar)
    HBsigma = ma.masked_array(HBsigma, HBmask)
    
    
    Ha_center = list(abs(wave-6562.79)).index(min(abs(wave - 6562.79)))
    Hamask = mask[Ha_center - 300:Ha_center + 300]
    Haroi = ma.masked_array(wave[Ha_center - 300:Ha_center + 300], Hamask)
    Haflux = ma.masked_array(flux[Ha_center - 300:Ha_center + 300], Hamask)
    Haivar = ivar[Ha_center - 300:Ha_center + 300]
    Hasigma = 1/np.sqrt(Haivar)
    Hasigma = ma.masked_array(Hasigma, Hamask)
    
    NIII_center = list(abs(wave-4200)).index(min(abs(wave - 4200)))
    NIIImask = mask[NIII_center - 250:NIII_center + 250]
    NIIIroi = ma.masked_array(wave[NIII_center- 250:NIII_center+250], NIIImask)
    NIIIflux = ma.masked_array(flux[NIII_center- 250:NIII_center+250], NIIImask)    
    NIIIivar = ivar[NIII_center - 250:NIII_center + 250]
    NIIIsigma = 1/np.sqrt(NIIIivar)
    NIIIsigma = ma.masked_array(NIIIsigma, NIIImask)
    
    try:
        HBopt, HBcov = curve_fit(HBgaus, HBroi[~HBroi.mask], HBflux[~HBflux.mask], \
                                 p0 = [1,3,1,5,5,0.125,5,0.125],sigma = HBsigma[~HBsigma.mask], \
                                 maxfev = 3000, absolute_sigma = True, check_finite = False)
    except (ValueError, RuntimeError) as e:
        HBopt = [1,1,1,1,1,1,1,1]
        HBcov = np.ones((8,8))
    
    try:
        Haopt, Hacov = curve_fit(Hagaus, Haroi[~Haroi.mask], Haflux[~Haflux.mask], \
                             p0 = [3,5,1,1],sigma = Hasigma[~Hasigma.mask], \
                                 maxfev = 3000, absolute_sigma = True, check_finite = False)
    except (ValueError, RuntimeError) as e:
        Haopt = [1,1,1,1]
        Hacov = np.ones((4,4))
    try:
        NIIIopt, NIIIcov = curve_fit(NIIIgaus, NIIIroi[~NIIIroi.mask], NIIIflux[~NIIIflux.mask], \
                         p0 = [1,2,1,3],sigma = NIIIsigma[~NIIIsigma.mask], \
                                     maxfev = 3000,absolute_sigma = True, check_finite = False)
    except (ValueError, RuntimeError) as e:
        NIIIopt = [1,1,1,1]
        NIIIcov = np.ones((4,4))


    
    
    Haexp = Hagaus(Haroi, *Haopt)
    rHa = Haflux - Haexp
    Hachisq = np.sum(rHa**2/Hasigma**2)
    Hachisq = Hachisq/(len(Haroi)-len(Haopt))
    
    HBexp = HBgaus(HBroi, *HBopt)
    rHB = HBflux - HBexp
    HBchisq = np.sum(rHB**2/HBsigma**2)
    HBchisq = HBchisq/(len(HBroi)-len(HBopt))
    
    
    NIIIexp = NIIIgaus(NIIIroi, *NIIIopt)
    rNIII = NIIIflux - NIIIexp
    NIIIchisq = np.sum(rNIII**2/NIIIsigma**2)
    NIIIchisq = NIIIchisq/(len(NIIIroi)-len(NIIIopt))
    
    Hacov = abs(Hacov)
    HBcov = abs(HBcov)
    NIIIcov = abs(NIIIcov)
    heights = []
    heights_err = []
    means = []
    means_err = []
    sigmas = []
    sigmas_err = []
    vs = []
    chi2s = []
    
    heights.append(Haopt[0]) #Ha data
    # means.append(Haopt[1])
    sigmas.append(Haopt[1])
    vs.append((Haopt[1]*2.355/6562)*c)
    heights_err.append(np.sqrt(Hacov[0][0]))
    # means_err.append(np.sqrt(Hacov[1][1]))
    sigmas_err.append(np.sqrt(Hacov[1][1]))
    chi2s.append(Hachisq)
    
    heights.append(HBopt[0]) #HB data
    # means.append(HBopt[4])
    sigmas.append(HBopt[1])
    vs.append((HBopt[1]*2.355/4861)*c)
    heights_err.append(np.sqrt(HBcov[0][0]))
    # means_err.append(np.sqrt(HBcov[4][4]))
    sigmas_err.append(np.sqrt(HBcov[1][1]))
    chi2s.append(HBchisq)
    
    heights.append(NIIIopt[2]) #Hgamma data
    # means.append(NIIIopt[4])
    sigmas.append(NIIIopt[3])
    vs.append((NIIIopt[3]*2.355/4340)*c)
    heights_err.append(np.sqrt(NIIIcov[2][2]))
    # # means_err.append(np.sqrt(NIIIcov[4][4]))
    sigmas_err.append(np.sqrt(NIIIcov[3][3]))
    chi2s.append(NIIIchisq)
         
    heights.append(HBopt[2]) #HeII4686 data
    # means.append(HBopt[1])
    sigmas.append(HBopt[3])
    vs.append((HBopt[3]*2.355/4686)*c)
    heights_err.append(np.sqrt(HBcov[2][2]))
    # means_err.append(np.sqrt(HBcov[1][1]))
    sigmas_err.append(np.sqrt(HBcov[3][3]))
    chi2s.append(HBchisq)
    
    heights.append(HBopt[4]) #OIII5007 data
    # means.append(HBopt[7])
    sigmas.append(HBopt[5])
    vs.append((HBopt[5]/5007)*c)
    heights_err.append(np.sqrt(HBcov[4][4]))
    # means_err.append(np.sqrt(HBcov[7][7]))
    sigmas_err.append(np.sqrt(HBcov[5][5]))   
    chi2s.append(HBchisq)

    heights.append(NIIIopt[0])#NIII data
    # means.append(NIIIopt[1])
    sigmas.append(NIIIopt[1])
    vs.append((NIIIopt[1]*2.355/4100)*c)
    heights_err.append(np.sqrt(NIIIcov[0][0]))
    # means_err.append(np.sqrt(NIIIcov[1][1]))
    sigmas_err.append(np.sqrt(NIIIcov[2][2]))
    chi2s.append(NIIIchisq)
    
    heights.append(Haopt[2]) #SII data
    # means.append(Haopt[4])
    sigmas.append(Haopt[3])
    vs.append((Haopt[3]*2.355/6732)*c)
    heights_err.append(np.sqrt(Hacov[2][2]))
    # means_err.append(np.sqrt(Hacov[2][2]))
    sigmas_err.append(np.sqrt(Hacov[3][3]))
    chi2s.append(Hachisq)
    
    heights.append(HBopt[6]) #OIII4959data
    # means.append(HBopt[10])
    sigmas.append(HBopt[7])
    vs.append((HBopt[7]/4959)*c)
    heights_err.append(np.sqrt(HBcov[6][6]))
    # means_err.append(np.sqrt(HBcov[10][10]))
    sigmas_err.append(np.sqrt(HBcov[7][7]))   
    chi2s.append(HBchisq)

    
    
    sigmas = [abs(i) for i in sigmas]
    vs = [abs(i) for i in vs]
    linetable = Table([lines,restwavelengths,heights,heights_err,sigmas,sigmas_err,\
                  vs,chi2s],names = ('Line','Wavelength','Height','e_Height','Sigma',\
                               'e_Sigma','Velocity','Chi Square'))
    linetable = linetable.to_pandas()
    return linetable

In [90]:
#def line_finder(wave, flux,ivar,mask,z):
#dif_spectra.wave, dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, zinfo
c = 2.99e5 #km/s

lines = ['Halpha','Hbeta', 'Hgamma','HeII4686','OIII5007','NIII','SII','OIII4959']
restwavelengths = [6562, 4861, 4340, 4686, 5007, 4100, 6732, 4959]

key = list(dif_spectra.wave.keys())[0] #if data is coadded, should be the first key
print(key)
print(dif_spectra.wave[key])
wave, flux, mask, ivar = dif_spectra.wave[key], dif_spectra.flux[key][0], dif_spectra.mask[key][0], dif_spectra.ivar[key][0]
print(abs(wave - 4861.4))
print(min(abs(wave - 4861.4)))
HB_center = list(abs(wave - 4861.4)).index(min(abs(wave - 4861.4)))
print(HB_center)
print(wave, flux, mask, ivar)

brz
[3600.  3600.8 3601.6 ... 9822.4 9823.2 9824. ]
[1261.4 1260.6 1259.8 ... 4961.  4961.8 4962.6]
0.20000000028721843
1577
[3600.  3600.8 3601.6 ... 9822.4 9823.2 9824. ] [-1.42623704 -1.36372615 -0.12953845 ...  0.15670493  0.12701888
  0.05652314] [0 0 0 ... 0 0 0] [ 0.03366957  0.05249168  0.05169035 ...  3.96493107  6.26921934
 11.54457225]


In [95]:
HBmask = mask[HB_center - 400:HB_center + 400]
print(HBmask[0])

HBroi = ma.masked_array(wave[HB_center - 400:HB_center + 400], HBmask)
print(HBroi[0])

HBflux = ma.masked_array(flux[HB_center - 400:HB_center + 400], HBmask)
print(HBflux[0])

HBivar = ivar[HB_center - 400:HB_center + 400]
print(HBivar[0])

HBsigma = 1/np.sqrt(HBivar)
print(HBsigma[0])

HBsigma = ma.masked_array(HBsigma, HBmask)
print(HBsigma[0])

0
4541.600000000214
-0.13795701619652845
1.430518869917547
0.8360903377067204
0.8360903377067204


In [97]:

Ha_center = list(abs(wave-6562.79)).index(min(abs(wave - 6562.79)))
Hamask = mask[Ha_center - 300:Ha_center + 300]
Haroi = ma.masked_array(wave[Ha_center - 300:Ha_center + 300], Hamask)
Haflux = ma.masked_array(flux[Ha_center - 300:Ha_center + 300], Hamask)
Haivar = ivar[Ha_center - 300:Ha_center + 300]
Hasigma = 1/np.sqrt(Haivar)
Hasigma = ma.masked_array(Hasigma, Hamask)

NIII_center = list(abs(wave-4200)).index(min(abs(wave - 4200)))
NIIImask = mask[NIII_center - 250:NIII_center + 250]
NIIIroi = ma.masked_array(wave[NIII_center- 250:NIII_center+250], NIIImask)
NIIIflux = ma.masked_array(flux[NIII_center- 250:NIII_center+250], NIIImask)    
NIIIivar = ivar[NIII_center - 250:NIII_center + 250]
NIIIsigma = 1/np.sqrt(NIIIivar)
NIIIsigma = ma.masked_array(NIIIsigma, NIIImask)

try:
    HBopt, HBcov = curve_fit(HBgaus, HBroi[~HBroi.mask], HBflux[~HBflux.mask], \
                             p0 = [1,3,1,5,5,0.125,5,0.125],sigma = HBsigma[~HBsigma.mask], \
                             maxfev = 3000, absolute_sigma = True, check_finite = False)
except (ValueError, RuntimeError) as e:
    HBopt = [1,1,1,1,1,1,1,1]
    HBcov = np.ones((8,8))

try:
    Haopt, Hacov = curve_fit(Hagaus, Haroi[~Haroi.mask], Haflux[~Haflux.mask], \
                         p0 = [3,5,1,1],sigma = Hasigma[~Hasigma.mask], \
                             maxfev = 3000, absolute_sigma = True, check_finite = False)
except (ValueError, RuntimeError) as e:
    Haopt = [1,1,1,1]
    Hacov = np.ones((4,4))
try:
    NIIIopt, NIIIcov = curve_fit(NIIIgaus, NIIIroi[~NIIIroi.mask], NIIIflux[~NIIIflux.mask], \
                     p0 = [1,2,1,3],sigma = NIIIsigma[~NIIIsigma.mask], \
                                 maxfev = 3000,absolute_sigma = True, check_finite = False)
except (ValueError, RuntimeError) as e:
    NIIIopt = [1,1,1,1]
    NIIIcov = np.ones((4,4))




Haexp = Hagaus(Haroi, *Haopt)
rHa = Haflux - Haexp
Hachisq = np.sum(rHa**2/Hasigma**2)
Hachisq = Hachisq/(len(Haroi)-len(Haopt))

HBexp = HBgaus(HBroi, *HBopt)
rHB = HBflux - HBexp
HBchisq = np.sum(rHB**2/HBsigma**2)
HBchisq = HBchisq/(len(HBroi)-len(HBopt))


NIIIexp = NIIIgaus(NIIIroi, *NIIIopt)
rNIII = NIIIflux - NIIIexp
NIIIchisq = np.sum(rNIII**2/NIIIsigma**2)
NIIIchisq = NIIIchisq/(len(NIIIroi)-len(NIIIopt))

Hacov = abs(Hacov)
HBcov = abs(HBcov)
NIIIcov = abs(NIIIcov)
heights = []
heights_err = []
means = []
means_err = []
sigmas = []
sigmas_err = []
vs = []
chi2s = []

heights.append(Haopt[0]) #Ha data
# means.append(Haopt[1])
sigmas.append(Haopt[1])
vs.append((Haopt[1]*2.355/6562)*c)
heights_err.append(np.sqrt(Hacov[0][0]))
# means_err.append(np.sqrt(Hacov[1][1]))
sigmas_err.append(np.sqrt(Hacov[1][1]))
chi2s.append(Hachisq)

heights.append(HBopt[0]) #HB data
# means.append(HBopt[4])
sigmas.append(HBopt[1])
vs.append((HBopt[1]*2.355/4861)*c)
heights_err.append(np.sqrt(HBcov[0][0]))
# means_err.append(np.sqrt(HBcov[4][4]))
sigmas_err.append(np.sqrt(HBcov[1][1]))
chi2s.append(HBchisq)

heights.append(NIIIopt[2]) #Hgamma data
# means.append(NIIIopt[4])
sigmas.append(NIIIopt[3])
vs.append((NIIIopt[3]*2.355/4340)*c)
heights_err.append(np.sqrt(NIIIcov[2][2]))
# # means_err.append(np.sqrt(NIIIcov[4][4]))
sigmas_err.append(np.sqrt(NIIIcov[3][3]))
chi2s.append(NIIIchisq)

heights.append(HBopt[2]) #HeII4686 data
# means.append(HBopt[1])
sigmas.append(HBopt[3])
vs.append((HBopt[3]*2.355/4686)*c)
heights_err.append(np.sqrt(HBcov[2][2]))
# means_err.append(np.sqrt(HBcov[1][1]))
sigmas_err.append(np.sqrt(HBcov[3][3]))
chi2s.append(HBchisq)

heights.append(HBopt[4]) #OIII5007 data
# means.append(HBopt[7])
sigmas.append(HBopt[5])
vs.append((HBopt[5]/5007)*c)
heights_err.append(np.sqrt(HBcov[4][4]))
# means_err.append(np.sqrt(HBcov[7][7]))
sigmas_err.append(np.sqrt(HBcov[5][5]))   
chi2s.append(HBchisq)

heights.append(NIIIopt[0])#NIII data
# means.append(NIIIopt[1])
sigmas.append(NIIIopt[1])
vs.append((NIIIopt[1]*2.355/4100)*c)
heights_err.append(np.sqrt(NIIIcov[0][0]))
# means_err.append(np.sqrt(NIIIcov[1][1]))
sigmas_err.append(np.sqrt(NIIIcov[2][2]))
chi2s.append(NIIIchisq)

heights.append(Haopt[2]) #SII data
# means.append(Haopt[4])
sigmas.append(Haopt[3])
vs.append((Haopt[3]*2.355/6732)*c)
heights_err.append(np.sqrt(Hacov[2][2]))
# means_err.append(np.sqrt(Hacov[2][2]))
sigmas_err.append(np.sqrt(Hacov[3][3]))
chi2s.append(Hachisq)

heights.append(HBopt[6]) #OIII4959data
# means.append(HBopt[10])
sigmas.append(HBopt[7])
vs.append((HBopt[7]/4959)*c)
heights_err.append(np.sqrt(HBcov[6][6]))
# means_err.append(np.sqrt(HBcov[10][10]))
sigmas_err.append(np.sqrt(HBcov[7][7]))   
chi2s.append(HBchisq)



sigmas = [abs(i) for i in sigmas]
vs = [abs(i) for i in vs]
linetable = Table([lines,restwavelengths,heights,heights_err,sigmas,sigmas_err,\
              vs,chi2s],names = ('Line','Wavelength','Height','e_Height','Sigma',\
                           'e_Sigma','Velocity','Chi Square'))
linetable = linetable.to_pandas()
print(linetable)

/tmp/ipykernel_52176/845917901.py:6: RuntimeWarning: divide by zero encountered in true_divide
  Hasigma = 1/np.sqrt(Haivar)


       Line  Wavelength    Height  e_Height       Sigma    e_Sigma  \
0    Halpha        6562  0.053109       inf  114.193357        inf   
1     Hbeta        4861  1.000000  1.000000    1.000000   1.000000   
2    Hgamma        4340 -0.074375  0.375874    3.544044  21.578273   
3  HeII4686        4686  1.000000  1.000000    1.000000   1.000000   
4  OIII5007        5007  1.000000  1.000000    1.000000   1.000000   
5      NIII        4100 -0.138886  0.540920    2.708409   0.375874   
6       SII        6732 -0.066593       inf    0.015393        inf   
7  OIII4959        4959  1.000000  1.000000    1.000000   1.000000   

       Velocity  Chi Square  
0  12253.685014    0.023840  
1    144.855997    0.062057  
2    575.004847    0.030214  
3    150.265685    0.062057  
4     59.716397    0.062057  
5    465.149450    0.030214  
6      1.610079    0.023840  
7     60.294414    0.062057  


/global/common/software/desi/cori/desiconda/20211217-2.0.0/conda/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [16]:
# Search for signature lines of TDEs, only interested in Galaxies
linetable = test_line_finder(dif_spectra.wave, dif_flux_clipped, dif_spectra.ivar, dif_spectra.mask, zinfo)
# print(night, linetable)
#spectype = zinfo['SPECTYPE'][0]
Hline_score = Hline_filter(linetable)
# deriv_score = deriv_filter(dif_flux_clipped, dif_spectra.ivar,dif_spectra.mask)

#broadband
bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
narrowlinelogic = perres_filter_narrow >=2
broadlinelogic = perres_filter_broad >=3
# TDElogic = any([TDE_score == 2, TDE_score == 3, TDE_score == 4, TDE_score == 5])
Hlinelogic = any([Hline_score >= 1])
# derivlogic = any([deriv_score >= 3])

logic = [bblogic, narrowlinelogic,  broadlinelogic, Hlinelogic]
logic_name = ['Broadband', 'narrow line', 'broad line','Hline'] #must be in same order as logic!, use as mask
logic_name = numpy.ma.masked_array(logic_name, mask = [not i for i in logic])
#plt.clf()

/tmp/ipykernel_63434/3265042277.py:27: RuntimeWarning: divide by zero encountered in true_divide
  Hasigma = 1/np.sqrt(Haivar)
/global/common/software/desi/cori/desiconda/20211217-2.0.0/conda/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [1]:
plt.clf()
pdf = PdfPages(str(tile)+".pdf")

####if any(logic):
    #Uncomment next line if you want to print only those TargetIds that get plotted
#processed(t, tile, night)
fig = plt.figure()
#for b in dif_spectra.wave.keys():
### have hardcoded 'brz' in place of for b in keys()...
w=numpy.where(dif_spectra.mask['brz'] ==0)[0]

plt.plot(dif_spectra.wave['brz'][0],dif_spectra.flux['brz'][w],color='red', label = 'Difference')
plt.plot(current_spectra.wave['brz'][0],current_spectra.flux['brz'][w],color='blue',alpha=0.5, label = 'New Spectrum')
plt.plot(ref_spectra.wave['brz'][0],ref_spectra.flux['brz'][w],color='green',alpha=0.5, label = 'Reference Spectrum')
    
plt.legend()
#plt.xlim((lminb,lmaxz))
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux') 
plt.title(str(t) + "  " + str(night) + "  " + str(tile)  + "  " + str(logic_name))
plt.savefig(pdf, format = 'pdf')
plt.savefig(pdf1, format = 'pdf')
plt.close()

   

NameError: name 'plt' is not defined

In [57]:
print(dif_spectra.mask['brz'][w])
print(dif_spectra.wave['brz'])
print(dif_spectra.wave['brz'][w])
print(dif_spectra.wave[w])


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[3600.  3600.8 3601.6 ... 9822.4 9823.2 9824. ]
[3600. 3600. 3600. ... 3600. 3600. 3600.]


TypeError: unhashable type: 'numpy.ndarray'

In [22]:
 #SkyPortal functionality begins:

#SkyPortal's Id for the object
objID = 'DESI{}'.format(str(t))

#Code to check if this object already exists in SkyPortal. If not, create it

response = requests.get("https://desi-skyportal.lbl.gov/api/candidates/{}".format(objID), headers={"Authorization": f"token {token}"})
if response.status_code == 400:
    obj_data = {
            "ra": ra_dec_data['TARGET_RA'][0], #RA is required when creating a new object.
            "dec": ra_dec_data['TARGET_DEC'][0], #Same for DEC
            "id": objID,
            "redshift": zinfo,
            "filter_ids": [sp.filter_id(filter_name)],
            "passed_at": str(datetime.datetime.utcnow()) #UTC time when the object passed the filter
            }

    response = requests.post(
            "https://desi-skyportal.lbl.gov/api/candidates",
            json= obj_data,
            headers={"Authorization": f"token {token}"})

#Now, we send the differenced spectrum to SkyPortal
#First, check if the same spectrum already exists under the object
response = requests.get("https://desi-skyportal.lbl.gov/api/sources/{}/spectra".format(objID),headers={"Authorization": f"token {token}"})
post = True
# for i in range (len(response.json()['data']['spectra'])):
#     if (wavelengths == response.json()['data']['spectra'][i]['wavelengths'] and flux == response.json()['data']['spectra'][i]['fluxes']):
#         post = False
#Only if the exact same spectrum doesn't already exist, upload it.
if post:
    #Send difference spectrum to SP:
    diff_spectrum_data = {
            "obj_id": objID,
            "wavelengths": difwave['brz'].tolist(),
            "fluxes": difflux['brz'].tolist(),
            "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
            "origin": "DESIDIFF_everest", #Only the difference spectrum gets this tag in order to distinguish it on SkyPortal
            "group_ids": [sp.group_id("DESI")],
            "instrument_id": sp.instrument_id()
            }
    response = requests.post(
                '{}/api/spectrum'.format(sp.url),
                json= diff_spectrum_data,
                headers={"Authorization": f"token {token}"}) 

    #Send new spectra to SP:
    new_spectra_data = {
        "obj_id": objID,
        "wavelengths":newwave['brz'].tolist(),
        "fluxes": newflux['brz'].tolist(),
        "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
        "origin": "DESIDIFF_everest",
        "group_ids": [sp.group_id("DESI")],
        "instrument_id": sp.instrument_id()
        }
    response = requests.post(
            '{}/api/spectrum'.format(sp.url),
            json= new_spectra_data,
            headers={"Authorization": f"token {token}"}) 

    #Code to find the ref_night that is closest to new_night to be used as 'observed_at' for SP
    new_night = datetime.datetime(int(str(night)[:4]), int(str(night)[4:6]), int(str(night)[6:]))
    closest_night = datetime.datetime(int(str(ref_nights[0])[:4]), int(str(ref_nights[0])[4:6]), int(str(ref_nights[0])[6:]))
    for ref_night in ref_nights[1:]:
        d = datetime.datetime(int(str(ref_night)[:4]), int(str(ref_night)[4:6]), int(str(ref_night)[6:]))
        time_delta = new_night - d
        best = new_night - closest_night 
        if (time_delta.total_seconds() < best.total_seconds()):
            closest_night = d


    ref_spectra_data = {
        "obj_id": objID,
        "wavelengths": refwave['brz'].tolist(),
        "fluxes": refflux['brz'].tolist(),
        "observed_at": str(closest_night),
        "origin": "DESIDIFF_everest",
        "group_ids": [sp.group_id("DESI")],
        "instrument_id": sp.instrument_id()
        }
    response = requests.post(
            '{}/api/spectrum'.format(sp.url),
            json= ref_spectra_data,
            headers={"Authorization": f"token {token}"})    

pdf.close()
pdf1.close()

NameError: name 'ra_dec_data' is not defined